# ORGANIZADOR DE CARPETAS Y DOCS

A continuación se presentan dos ejemplos de bots que automatizan la gestión de archivos en un explorador de archivos cualquiera.

### Ejemplo 1: 

Código en python que organiza una serie de carpetas y archivos en función de los nombres de los mismos y la categoría que se les asigna en una hoja de excel.

### Ejemplo 2:

Código en VBA que hace una función similar, pero el funcionamiento del mismo se basa en una macro de excel. El usuario simplemente debe introducir un listado de nombres en excel, una ruta de destino y pulsar un botón. Organizando una serie de archivos y carpetas de forma más intuitiva y sin tener que ejecutar código personalmente.


## EJEMPLO 1: ARCHIVO PYTHON

In [ ]:
# Importación de librerías principales
import pandas as pd
import os
import re
import shutil

### Carga de Datos

In [ ]:
# Datos a cargar
df_datos=pd.read_excel('ruta/ARCHIVOS_TRABAJO/ORGANIZADOR CARPETAS Y DOCS//datos2.xlsx',sheet_name='Muestra',dtype=str)

# Ruta de la carpeta principal
ruta_principal = r'ruta/ARCHIVOS_TRABAJO/ORGANIZADOR CARPETAS Y DOCS/MUESTRA'

# Ruta de la carpeta con los documentos a mover
ruta_documentos = os.path.join('ruta/Desktop/DOCS MUESTRA')

### Movimiento de archivos

In [ ]:
# Crea la carpeta principal si no existe
if not os.path.exists(ruta_principal):
    os.mkdir(ruta_principal)

# Agrupa los datos por identificador y referencia
grupos = df_datos.groupby(['ZONA', 'Referencia'])

# Itera sobre los grupos y crea las carpetas correspondientes
for (zona, referencia), _ in grupos:
    # Crea la carpeta para el identificador si no existe
    carpeta_identificador = os.path.join(ruta_principal, zona)
    if not os.path.exists(carpeta_identificador):
        os.mkdir(carpeta_identificador)
    
    # Crea la carpeta para la referencia si no existe
    carpeta_referencia = os.path.join(carpeta_identificador, referencia)
    if not os.path.exists(carpeta_referencia):
        os.mkdir(carpeta_referencia)

In [ ]:
# Itera sobre los grupos y mueve los archivos correspondientes
for (zona, referencia), grupo in grupos:
    # Ruta de la carpeta correspondiente a la referencia
    carpeta_referencia = os.path.join(ruta_principal, zona, referencia)
    
    # Itera sobre los archivos en la carpeta de documentos
    for archivo in os.listdir(ruta_documentos):
        # Busca el número de referencia en el nombre del archivo
        if re.search(referencia, archivo):
            # Ruta completa del archivo a mover
            ruta_origen = os.path.join(ruta_documentos, archivo)
            
            # Ruta completa de destino del archivo
            ruta_destino = os.path.join(carpeta_referencia, archivo)
            
            # Mueve el archivo a la carpeta correspondiente
            shutil.move(ruta_origen, ruta_destino)

In [ ]:
#Carpetas vacias
# Lista de carpetas vacías
carpetas_vacias = []

# Itera sobre los grupos y verifica si las carpetas están vacías
for (zona, referencia), _ in grupos:
    # Ruta de la carpeta correspondiente a la referencia
    carpeta_referencia = os.path.join(ruta_principal, zona, referencia)
    
    # Verifica si la carpeta está vacía
    if not os.listdir(carpeta_referencia):
        carpetas_vacias.append(os.path.basename(carpeta_referencia))

# Imprime la lista de carpetas vacías
print(carpetas_vacias)

## EJEMPLO 2: CÓDIGO VBA
Este código es para una macro de Excel, cuyas instrucciones son las que se muestran a continuación:

![Captura de pantalla](./img/c1_.png)

![Captura de pantalla](./img/c2.png)

El aspecto de la hoja principal es este

![Captura de pantalla](./img/c3.png)

### CÓDIGO DE LA MACRO

Lo que hace es automatizar el proceso de compresión de archivos y su organización en carpetas separadas con convenciones de nombres específicas en Excel.

Este código VBA realiza las siguientes operaciones en un libro de Excel:

#### 1. **Compresión de Archivos en una Carpeta**
   - **Configuración de Rutas**: Se define la ruta al archivo `7z.exe` (el ejecutable de 7-Zip), así como la ruta de la carpeta (`rutaArchivo`) que se obtiene de la celda `C2` en la hoja "ORGANIZADOR".
   - **Compresión de Archivos**: La macro recorre cada archivo en la carpeta especificada y lo comprime en un archivo `.zip` utilizando 7-Zip. Crea un archivo ZIP con el mismo nombre que el archivo original (pero con extensión `.zip`) y lo guarda en el mismo directorio.

#### 2. **Esperar a que la Compresión se Complete**
   - La macro espera a que el proceso de 7-Zip termine antes de pasar al siguiente archivo, utilizando la función `WaitForProcessToComplete`.

#### 3. **Distribuir los Archivos Comprimidos en Carpetas Específicas**
   - La macro lee valores de la hoja "ORGANIZADOR" (empezando desde la fila 2) para obtener los nombres de las carpetas y la convención de nombres de los archivos.
   - Luego, verifica si existen carpetas con los nombres correspondientes a los valores en la columna A (por ejemplo, `codigo`). Si no existen, las crea.
   - Los archivos comprimidos `.zip` se copian a estas carpetas recién creadas, renombrándolos con un formato basado en el `codigo` (código), el `tipo` (valor de la celda `F6`), y un índice incremental.

#### Funciones y Detalles Claves:
   - **FolderExists**: Una función que verifica si una carpeta ya existe.
   - **FileExists**: Una función que verifica si un archivo existe en la ruta especificada.
   - **WaitForProcessToComplete**: Una función que espera a que termine el proceso de compresión de 7-Zip verificando el ID del proceso.

#### Manejo de Errores:
   - La macro utiliza `On Error GoTo ErrorHandler` para manejar cualquier error que pueda ocurrir durante el proceso, mostrando un mensaje de error si ocurre algún problema.

#### Resumen de los Pasos:
1. **Comprimir todos los archivos** en una carpeta utilizando 7-Zip.
2. **Distribuir los archivos comprimidos** en carpetas específicas según los valores de la hoja de Excel.
3. **Renombrar los archivos** durante el proceso de distribución con un formato basado en `codigo`, `tipo` y un índice.
4. **Manejo de Errores**: Si ocurre un error, se muestra un mensaje con la descripción del error.

In [ ]:
Sub ComprimirYDistribuir()

    Dim rutaArchivo As String
    Dim codigo As String
    Dim fila As Integer
    Dim path7Zip As String
    Dim shellCmd As String
    Dim zipPath As String
    Dim processID As Long
    Dim fso As Object, folder As Object, file As Object
    
    On Error GoTo ErrorHandler
    
    '1. Rutas e inicialización
    path7Zip = "C:\Program Files\7-Zip\7z.exe"
    rutaArchivo = ThisWorkbook.Sheets("ORGANIZADOR").Range("C2").Value
    
    Set fso = CreateObject("Scripting.FileSystemObject")
    Set folder = fso.GetFolder(rutaArchivo)
    
    Dim fileNameWithoutExtension As String
    
    '2. Comprimir todos los archivos en la carpeta
    For Each file In folder.Files
        ' Extrae el nombre del archivo sin su extensión
        fileNameWithoutExtension = Left(file.Name, InStrRev(file.Name, ".") - 1)
        
        zipPath = rutaArchivo & "\" & fileNameWithoutExtension & ".zip"
        shellCmd = """" & path7Zip & """ a """ & zipPath & """ """ & rutaArchivo & "\" & file.Name & """"
        
        processID = Shell(shellCmd, vbNormalFocus)
        
        ' Esperar hasta que el proceso de 7-Zip haya terminado
        WaitForProcessToComplete processID
        
    Next file

    
    'Refresca la carpeta
    Set folder = fso.GetFolder(rutaArchivo)

    
    '3. Distribuimos cada archivo ZIP a las carpetas correspondientes
    Dim tipo As String
    
    tipo = ThisWorkbook.Sheets("ORGANIZADOR").Range("F6").Value
    fila = 2
    
    Do While ThisWorkbook.Sheets("ORGANIZADOR").Cells(fila, 1).Value <> ""
        Dim indice As Integer 'Mueve la declaración de índice aquí para resetearlo con cada carpeta
        indice = 1 ' Inicializa el índice para cada carpeta
    
        codigo = ThisWorkbook.Sheets("ORGANIZADOR").Cells(fila, 1).Value
    
        ' Crear carpeta con el nombre del código si no existe
        If Not FolderExists(rutaArchivo & "\" & codigo) Then
            MkDir rutaArchivo & "\" & codigo
        End If
    
        ' Copiar los archivos ZIP a la carpeta del código
        For Each file In folder.Files
            ' Comprobamos si el archivo es un archivo .zip
            If Right(file.Name, 4) = ".zip" Then
                If FileExists(rutaArchivo & "\" & file.Name) Then
                    Dim nuevoNombre As String
                    nuevoNombre = codigo & "_" & tipo & "_" & indice & ".zip"
                    FileCopy rutaArchivo & "\" & file.Name, rutaArchivo & "\" & codigo & "\" & nuevoNombre
                    indice = indice + 1
                Else
                    MsgBox "El archivo " & rutaArchivo & "\" & file.Name & " no existe.", vbExclamation, "Error"
                    Exit Sub
                End If
            End If
        Next file
    
        fila = fila + 1
    Loop
    
    MsgBox "Proceso terminado, refresca la carpeta para ver los resultados", vbInformation, "Información"

Exit Sub
ErrorHandler:
MsgBox "Se ha producido un error: " & Err.Description, vbCritical, "Error"

End Sub

Function FolderExists(ByVal FolderPath As String) As Boolean
    On Error Resume Next
    FolderExists = (GetAttr(FolderPath) And vbDirectory) = vbDirectory
    On Error GoTo 0
End Function

Function FileExists(ByVal FilePath As String) As Boolean
    On Error Resume Next
    FileExists = Len(Dir(FilePath)) > 0
    On Error GoTo 0
End Function


Sub WaitForProcessToComplete(processID As Long)
    Dim objWMIService, objProcess, colProcess
    Dim strComputer, strList
    Dim blnRunning As Boolean

    blnRunning = True
    strComputer = "."

    Application.Wait (Now + TimeValue("0:00:05")) ' Espera 5 segundos antes de verificar si el proceso ha terminado

    Set objWMIService = GetObject("winmgmts:" & "{impersonationLevel=impersonate}!\\" & strComputer & "\root\cimv2")
    
    While blnRunning
        blnRunning = False
        Set colProcess = objWMIService.ExecQuery("Select * from Win32_Process where ProcessID =" & processID)
        For Each objProcess In colProcess
            blnRunning = True
        Next
        DoEvents
    Wend
End Sub
